***Data Cleaning and Preparing***

In [1]:
# Importing essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load the dataset from the Excel file
df = pd.read_excel("D:/GOKLYN INTERNSHIP 2025/3. Project 2/Clean Data/cd.xlsx")

# Display the first 5 rows of the dataset
df.head()

,continent,country,date,population,total_cases,total_cases_per_million,new_cases,new_cases_smoothed,total_deaths,total_deaths_per_million,...,total_tests,new_tests,positive_rate,tests_per_case,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,population_density
0,Asia,Afghanistan,2020-01-04,40578801.0,0,0.0,0.0,NaN,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.215477
1,Asia,Afghanistan,2020-01-05,40578801.0,0,0.0,0.0,NaN,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.215477
2,Asia,Afghanistan,2020-01-06,40578801.0,0,0.0,0.0,NaN,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.215477
3,Asia,Afghanistan,2020-01-07,40578801.0,0,0.0,0.0,NaN,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.215477
4,Asia,Afghanistan,2020-01-08,40578801.0,0,0.0,0.0,NaN,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.215477


In [3]:
# Displaying the shape (number of rows and columns) of the dataset to understand its size.
df.shape

(486795, 23)

In [4]:
# Getting a concise summary of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486795 entries, 0 to 486794
Data columns (total 23 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   continent                  455515 non-null  object        
 1   country                    486795 non-null  object        
 2   date                       486795 non-null  datetime64[ns]
 3   population                 477020 non-null  float64       
 4   total_cases                486795 non-null  int64         
 5   total_cases_per_million    486795 non-null  float64       
 6   new_cases                  483980 non-null  float64       
 7   new_cases_smoothed         482764 non-null  float64       
 8   total_deaths               486795 non-null  int64         
 9   total_deaths_per_million   486795 non-null  float64       
 10  new_deaths                 484741 non-null  float64       
 11  new_deaths_smoothed        483529 non-null  float64 

In [5]:
# Convert categorical/object columns to string type
df['continent'] = df['continent'].astype('string')
df['country'] = df['country'].astype('string')

# Columns expected to be whole numbers but may contain NaNs
int_cols = [
    'population', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths',
    'total_tests', 'new_tests',
    'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated',
    'new_vaccinations'
]

# Round and convert to nullable integer
for col in int_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].round().astype('Int64')

# Columns that should remain float (ratios, rolling averages, densities)
float_cols = [
    'total_cases_per_million', 'new_cases_smoothed',
    'total_deaths_per_million', 'new_deaths_smoothed',
    'stringency_index', 'tests_per_case', 'positive_rate',
    'new_vaccinations_smoothed', 'population_density'
]

for col in float_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('float64')


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486795 entries, 0 to 486794
Data columns (total 23 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   continent                  455515 non-null  string        
 1   country                    486795 non-null  string        
 2   date                       486795 non-null  datetime64[ns]
 3   population                 477020 non-null  Int64         
 4   total_cases                486795 non-null  Int64         
 5   total_cases_per_million    486795 non-null  float64       
 6   new_cases                  483980 non-null  Int64         
 7   new_cases_smoothed         482764 non-null  float64       
 8   total_deaths               486795 non-null  Int64         
 9   total_deaths_per_million   486795 non-null  float64       
 10  new_deaths                 484741 non-null  Int64         
 11  new_deaths_smoothed        483529 non-null  float64 

In [7]:
# Check for missing (null) values in each column of the DataFrame
df.isnull().sum()

continent                     31280
country                           0
date                              0
population                     9775
total_cases                       0
total_cases_per_million           0
new_cases                      2815
new_cases_smoothed             4031
total_deaths                      0
total_deaths_per_million          0
new_deaths                     2054
new_deaths_smoothed            3266
stringency_index             288962
total_tests                  408994
new_tests                    412877
positive_rate                385236
tests_per_case               386151
total_vaccinations           405812
people_vaccinated            409915
people_fully_vaccinated      411768
new_vaccinations             419812
new_vaccinations_smoothed    287637
population_density            17595
dtype: int64

In [8]:
# List of known non-country entries that appear in the dataset.
# These represent regions or income groups, not individual countries.
non_countries = [
    'World', 'Asia', 'Africa', 'Europe', 'Oceania', 'North America', 'South America',
    'European Union (27)', 'High-income countries', 'Low-income countries',
    'Upper-middle-income countries', 'Lower-middle-income countries',
    'World excl. China', 'World excl. China and South Korea',
    'World excl. China, South Korea, Japan and Singapore',
    'Asia excl. China'
]

# Filter the DataFrame to exclude rows where the 'country' column
# matches any entry in the non_countries list
df = df[~df['country'].isin(non_countries)]

# Reset the DataFrame index after filtering to maintain a clean index
df.reset_index(drop=True, inplace=True)

In [9]:
df.isnull().sum()

continent                         0
country                           0
date                              0
population                     1955
total_cases                       0
total_cases_per_million           0
new_cases                      2815
new_cases_smoothed             3951
total_deaths                      0
total_deaths_per_million          0
new_deaths                     2054
new_deaths_smoothed            3186
stringency_index             257682
total_tests                  377714
new_tests                    381597
positive_rate                353956
tests_per_case               354871
total_vaccinations           390732
people_vaccinated            394835
people_fully_vaccinated      396688
new_vaccinations             404732
new_vaccinations_smoothed    272557
population_density             9775
dtype: int64

In [10]:
# Drop rows from the DataFrame where the 'population' column has missing (NaN) values
df = df.dropna(subset=['population'])

In [11]:
# Count the number of missing (NaN) values specifically in the 'population' column
df['population'].isnull().sum()

0

In [12]:
# Replace all missing (NaN) values in the 'new_cases' column with 0
df['new_cases'] = df['new_cases'].fillna(0)

# Count how many missing values are left in the 'new_cases' column (should be 0 after filling)
df['new_cases'].isnull().sum()

0

In [13]:
# Replace all missing (NaN) values in the 'new_cases_smoothed' column with 0
df['new_cases_smoothed'] = df['new_cases_smoothed'].fillna(0)

# Count and display the number of missing values remaining in the 'new_cases_smoothed' column
# (should return 0 if all NaNs were successfully filled)
df['new_cases_smoothed'].isnull().sum()

0

In [14]:
# Replace all missing (NaN) values in the 'new_deaths' column with 0
df['new_deaths'] = df['new_deaths'].fillna(0)

# Count and display the number of remaining missing values in the 'new_deaths' column
# (expected to be 0 after filling)
df['new_deaths'].isnull().sum()

0

In [15]:
# Replace all missing (NaN) values in the 'new_deaths_smoothed' column with 0
df['new_deaths_smoothed'] = df['new_deaths_smoothed'].fillna(0)

# Count and display the number of remaining missing values in the 'new_deaths_smoothed' column
# (should be 0 after filling)
df['new_deaths_smoothed'].isnull().sum()

0

In [16]:
# Forward-fill missing values in the 'stringency_index' column
# This means each NaN is replaced with the last valid value seen above it
df['stringency_index'] = df['stringency_index'].fillna(method='ffill')

C:\Users\Asus\AppData\Local\Temp\ipykernel_25620\3281007841.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['stringency_index'] = df['stringency_index'].fillna(method='ffill')


In [17]:
# Count and display the number of remaining missing (NaN) values in the 'stringency_index' column
df['stringency_index'].isnull().sum()

0

In [18]:
# Replace all missing (NaN) values in the 'new_tests' column with 0
df['new_tests'] = df['new_tests'].fillna(0)

In [19]:
# Count and display the number of remaining missing (NaN) values in the 'new_tests' column
df['new_tests'].isnull().sum()

0

In [20]:
# First, sort the DataFrame by 'country' and then by 'date'
# This ensures that forward-filling operates on chronologically ordered data within each country
df = df.sort_values(by=['country', 'date'])

# Forward-fill missing values in the 'total_tests' column, separately within each country
# This assumes the total number of tests remains the same until updated with a new value
df['total_tests'] = df.groupby('country')['total_tests'].ffill()

In [21]:
# Backward-fill missing values in the 'total_tests' column within each country group
# This replaces NaN values with the next valid value that appears below in the time series
df['total_tests'] = df.groupby('country')['total_tests'].bfill()

In [22]:
# Identify countries that have only missing values (NaN) in the 'total_tests' column
# These countries never reported any testing data
no_test_countries = df[df['total_tests'].isna()]['country'].unique()

# Remove all rows belonging to those countries from the DataFrame
df = df[~df['country'].isin(no_test_countries)]

In [23]:
# Check and display the total number of missing (NaN) values in each column of the DataFrame
df.isnull().sum()

continent                         0
country                           0
date                              0
population                        0
total_cases                       0
total_cases_per_million           0
new_cases                         0
new_cases_smoothed                0
total_deaths                      0
total_deaths_per_million          0
new_deaths                        0
new_deaths_smoothed               0
stringency_index                  0
total_tests                       0
new_tests                         0
positive_rate                259685
tests_per_case               260600
total_vaccinations           299314
people_vaccinated            303123
people_fully_vaccinated      304894
new_vaccinations             311524
new_vaccinations_smoothed    201581
population_density             1955
dtype: int64

In [24]:
# Count the number of rows where the value in the 'new_tests' column is exactly 0
(df['new_tests'] == 0).sum()

288065

In [25]:
# Forward-fill missing values in the 'new_tests' column within each country group
# This assumes that the number of new tests remains the same as the last reported value
df['new_tests'] = df.groupby('country')['new_tests'].ffill()

In [26]:
# Calculate the COVID-19 test positivity rate for each row
# Divide 'new_cases' by 'new_tests' to get the fraction of tests that are positive
# Replace 0s in 'new_tests' with NaN to avoid division by zero errors
df['positive_rate'] = df['new_cases'] / df['new_tests'].replace(0, np.nan)

In [27]:
# Count and display the number of missing (NaN) values in the 'positive_rate' column
df['positive_rate'].isnull().sum()

288065

In [28]:
# Calculate the number of tests conducted per confirmed case
# Divide 'total_tests' by 'total_cases'
# Replace 0 in 'total_cases' with NaN to avoid division by zero
df['tests_per_case'] = df['total_tests'] / df['total_cases'].replace(0, np.nan)

In [29]:
# Count and display the number of missing (NaN) values remaining in the 'tests_per_case' column
df['tests_per_case'].isnull().sum()

14214

In [30]:
# Fill missing values in the 'tests_per_case' column within each country group
# First, forward-fill (ffill) missing values — propagating the last known value downward
# Then, backward-fill (bfill) any remaining NaNs — propagating the next known value upward
df['tests_per_case'] = df.groupby('country')['tests_per_case'].ffill().bfill()

In [31]:
# Count and display the number of missing (NaN) values remaining in the 'tests_per_case' column
df['tests_per_case'].isnull().sum()

0

In [32]:
# Define a list of vaccination-related columns to clean
vax_cols = [
    'total_vaccinations',           # Cumulative total number of vaccine doses administered
    'people_vaccinated',            # Number of people who received at least one dose
    'people_fully_vaccinated',      # Number of people who received all required doses
    'new_vaccinations',             # Number of vaccinations administered on a given day
    'new_vaccinations_smoothed'     # Smoothed version (e.g., 7-day average) of new vaccinations
]

# For each vaccination-related column, fill missing values within each country's time series
for col in vax_cols:
    # First forward-fill to propagate previous known values
    df[col] = df.groupby('country')[col].ffill()
    
    # Then backward-fill to fill in missing values at the beginning of the time series
    df[col] = df.groupby('country')[col].bfill()

In [33]:
# Display the number of missing (NaN) values for each column in the DataFrame
df.isnull().sum()

continent                         0
country                           0
date                              0
population                        0
total_cases                       0
total_cases_per_million           0
new_cases                         0
new_cases_smoothed                0
total_deaths                      0
total_deaths_per_million          0
new_deaths                        0
new_deaths_smoothed               0
stringency_index                  0
total_tests                       0
new_tests                         0
positive_rate                288065
tests_per_case                    0
total_vaccinations            15640
people_vaccinated             15640
people_fully_vaccinated       19550
new_vaccinations              62560
new_vaccinations_smoothed     15640
population_density             1955
dtype: int64

In [34]:
# Identify countries that have at least one non-null value in 'total_vaccinations'
# This checks if the country has reported any vaccination data at all
countries_with_vax = df.groupby('country')['total_vaccinations'].apply(lambda x: x.notna().any())

# Extract the list of country names where vaccination data exists (True values)
valid_countries = countries_with_vax[countries_with_vax].index

# Filter the DataFrame to only include countries with vaccination data
df = df[df['country'].isin(valid_countries)]

In [35]:
# Count and display the number of unique countries remaining in the dataset
df['country'].nunique()

176

In [36]:
# Display the number of missing (NaN) values in the last 8 columns of the DataFrame
df.isnull().sum().tail(8)

positive_rate                275521
tests_per_case                    0
total_vaccinations                0
people_vaccinated                 0
people_fully_vaccinated        3910
new_vaccinations              46920
new_vaccinations_smoothed         0
population_density             1955
dtype: int64

In [37]:
# Identify countries that have at least one non-null entry in 'total_vaccinations'
# This ensures we only keep countries that have reported some vaccination data
countries_with_vax_data = df.groupby('country')['total_vaccinations'].apply(lambda x: x.notna().any())

# Filter the DataFrame to retain only those countries
# Extract country names where the condition is True (i.e., they have vaccination data)
df = df[df['country'].isin(countries_with_vax_data[countries_with_vax_data].index)]

In [38]:
# Show the number of missing values in the last 8 columns of the DataFrame
df.isnull().sum().tail(8)

positive_rate                275521
tests_per_case                    0
total_vaccinations                0
people_vaccinated                 0
people_fully_vaccinated        3910
new_vaccinations              46920
new_vaccinations_smoothed         0
population_density             1955
dtype: int64

In [39]:
# Fill missing values in 'population_density' for each country
# First, forward-fill to carry the last known value forward
# Then, backward-fill to handle missing values at the start of each country's data
df['population_density'] = df.groupby('country')['population_density'].ffill().bfill()

In [40]:
# Count and display the number of missing (NaN) values in the 'population_density' column
df['population_density'].isnull().sum()

0

In [41]:
# Display the total number of missing (NaN) values for each column in the DataFrame
df.isnull().sum()

continent                         0
country                           0
date                              0
population                        0
total_cases                       0
total_cases_per_million           0
new_cases                         0
new_cases_smoothed                0
total_deaths                      0
total_deaths_per_million          0
new_deaths                        0
new_deaths_smoothed               0
stringency_index                  0
total_tests                       0
new_tests                         0
positive_rate                275521
tests_per_case                    0
total_vaccinations                0
people_vaccinated                 0
people_fully_vaccinated        3910
new_vaccinations              46920
new_vaccinations_smoothed         0
population_density                0
dtype: int64

In [42]:
# Fill missing values in 'people_fully_vaccinated' column by using both forward and backward fill within each country
# Forward fill: carries the last known value forward
# Backward fill: fills in values at the start of a country's data where forward fill can't help
df['people_fully_vaccinated'] = df.groupby('country')['people_fully_vaccinated'].ffill().bfill()

In [43]:
# Count and display how many missing (NaN) values remain in the 'people_fully_vaccinated' column
df['people_fully_vaccinated'].isnull().sum()

0

In [44]:
# Interpolate missing values in the 'new_vaccinations' column within each country
# This assumes vaccinations continued even if data wasn't reported on some days
# 'transform' ensures the result is aligned with the original DataFrame index
df['new_vaccinations'] = df.groupby('country')['new_vaccinations'].transform(
    lambda x: x.interpolate(limit_direction='both')
)

In [45]:
# Check for remaining NaNs
df['new_vaccinations'].isnull().sum()

46920

In [46]:
# Fill any remaining missing values in 'new_vaccinations' with 0
# This assumes that if data is still missing after interpolation, no vaccinations occurred that day
df['new_vaccinations'] = df['new_vaccinations'].fillna(0)

# Clip any negative values (from interpolation errors) to 0
# Vaccination counts can't be negative
df['new_vaccinations'] = df['new_vaccinations'].clip(lower=0)

In [47]:
# Display the total number of missing (NaN) values in each column of the DataFrame
df.isnull().sum()

continent                         0
country                           0
date                              0
population                        0
total_cases                       0
total_cases_per_million           0
new_cases                         0
new_cases_smoothed                0
total_deaths                      0
total_deaths_per_million          0
new_deaths                        0
new_deaths_smoothed               0
stringency_index                  0
total_tests                       0
new_tests                         0
positive_rate                275521
tests_per_case                    0
total_vaccinations                0
people_vaccinated                 0
people_fully_vaccinated           0
new_vaccinations                  0
new_vaccinations_smoothed         0
population_density                0
dtype: int64

In [48]:
# Show only the columns with remaining missing values
df.isnull().sum()[df.isnull().sum() > 0]

positive_rate    275521
dtype: int64

In [49]:
# Interpolate missing values in 'positive_rate' within each country to estimate plausible values
# limit_direction='both' fills NaNs at the start and end of each country's time series as well
df['positive_rate'] = df.groupby('country')['positive_rate'].transform(
    lambda x: x.interpolate(limit_direction='both')
)


# “Plausible” means something that makes sense or is reasonable given the context — not guaranteed to be exactly true,
# but believable based on the data we have.

In [50]:
# Count how many missing (NaN) values remain in the 'positive_rate' column after interpolation
df['positive_rate'].isnull().sum()

66470

In [51]:
# Fill any remaining missing values in 'positive_rate' with 0,
# assuming no positive cases when data is missing
df['positive_rate'] = df['positive_rate'].fillna(0)

# Ensure all values in 'positive_rate' lie between 0 and 1 (valid ratio range)
df['positive_rate'] = df['positive_rate'].clip(lower=0, upper=1)

In [52]:
df.isnull().sum()

continent                    0
country                      0
date                         0
population                   0
total_cases                  0
total_cases_per_million      0
new_cases                    0
new_cases_smoothed           0
total_deaths                 0
total_deaths_per_million     0
new_deaths                   0
new_deaths_smoothed          0
stringency_index             0
total_tests                  0
new_tests                    0
positive_rate                0
tests_per_case               0
total_vaccinations           0
people_vaccinated            0
people_fully_vaccinated      0
new_vaccinations             0
new_vaccinations_smoothed    0
population_density           0
dtype: int64